In [79]:
!pip install pyspark



In [80]:
!pip install Koalas
!pip install bokeh


In [81]:
import numpy as np
import pandas as pd
from datetime import datetime
from bokeh.palettes import Spectral4 #4가지 선그래프 사용가능한 고정폼
from bokeh.plotting import figure, show, output_file,save
from bokeh.io import output_notebook, curdoc
# from bokeh.models import *
from bokeh.themes import Theme
from bokeh.layouts import column, layout


output_notebook()

curdoc().theme = Theme(filename="custom_bokeh_theme.yml")
format = '%Y-%m-%d'


Loading BokehJS ...

In [82]:
bb = pd.read_parquet('second.parquet/confirmed_cases.parquet')
cc = pd.read_parquet('second.parquet/metro_per_week.parquet')
#폼에 넣기 위해 컬럼명 변경
cc.rename(columns={'사용일자':'주간시작일자'},inplace =True)
cc.rename(columns={'승차총승객수':'신규확진자'},inplace =True)

def fun(x):
    return x*2000 + 22000000

bb["신규확진자"] = bb["신규확진자"].apply(fun)

In [83]:

p = figure(width=800, height=500, x_axis_type="datetime", toolbar_location=None)

def date(strs):
    return datetime.strptime(strs,format)
#박스를 적용 해봅시다
"""20.3.22~5.5	5단계(사회적거리두기)
5.6~8.15	1단계
8.16~8.29	3단계
8.16~9.13	4단계
9.14~10.11	3단계
10.12~11.23	1단계
11.24~12.7	3단계
12.8~21.2.14	4단계
2.15~7.11	3단계"""

def viewBox(value):
    if value:
        print(value)
        box = BoxAnnotation(left=date("2020-03-22"), right=date("2020-05-05"),fill_alpha=0.45, fill_color='red')
        p.add_layout(box)
        box = BoxAnnotation(left=date("2020-05-05"), right=date("2020-08-15"),fill_alpha=0.18, fill_color='red')
        p.add_layout(box)
        box = BoxAnnotation(left=date("2020-08-15"), right=date("2020-08-29"),fill_alpha=0.3, fill_color='red')
        p.add_layout(box)
        box = BoxAnnotation(left=date("2020-08-29"), right=date("2020-09-13"),fill_alpha=0.4, fill_color='red')
        p.add_layout(box)
        box = BoxAnnotation(left=date("2020-09-13"), right=date("2020-10-11"),fill_alpha=0.3, fill_color='red')
        p.add_layout(box)
        box = BoxAnnotation(left=date("2020-10-11"), right=date("2020-11-23"),fill_alpha=0.18, fill_color='red')
        p.add_layout(box)
        box = BoxAnnotation(left=date("2020-11-23"), right=date("2020-12-07"),fill_alpha=0.3, fill_color='red')
        p.add_layout(box)
        box = BoxAnnotation(left=date("2020-12-07"), right=date("2021-02-14"),fill_alpha=0.4, fill_color='red')
        p.add_layout(box)
        box = BoxAnnotation(left=date("2021-02-14"), right=date("2021-09-20"),fill_alpha=0.3, fill_color='red')
        p.add_layout(box)
        toggle.active = True
    else:
        print(value)
        toggle.active = False

#버튼 만들기
from bokeh.io import show
from bokeh.models import CustomJS, Toggle
code = '''
if toggle.active
    box.fill_alpha = 0.1
    box.line_alpha = 1
    console.log("active")
else
    box.fill_alpha = 0
    box.line_alpha = 0
    console.log("inactive")
'''
#callback = CustomJS.from_coffeescript(code = code, args={})
#toggle = Toggle(label="asdfsdf", button_type="success", callback=callback)
# callback.args = {'toggle': toggle, 'box': box}
toggle = Toggle(label="거리두기단계표시",background= 'gray', button_type="success")
#toggle.js_on_click(CustomJS(code="""console.log('toggle: active=' + this.active, this.toString())"""))
toggle.js_on_click(CustomJS(code="console.log('ddsfdsfsfa')"))


#인터랙트 위젯 적용


p.extra_y_ranges = {"foo": Range1d(start=0, end=17000)}
p.add_layout(LinearAxis(y_range_name="foo",axis_line_color = '#FFFFFF',major_label_text_color = '#B2EBF4'), 'right')
p.yaxis.major_label_text_color = "#FFFFFF"
p.xaxis.major_label_text_color = "#FFFFFF"

p.x_range = DataRange1d(range_padding=0.0)
p.title.text = '승차-확진자 관련성'

zips = zip([bb, cc], ["주간산규확진자", "승차인원"], Spectral4)
palettess = ["#FFD1D1","#FFB4B4","#FFB2B2","#FF9999","#FF8C8C"]

color_mapper = LinearColorMapper(palette = palettess,low=0, high=5)

color_bar = ColorBar(title = "거리두기 단계(개정 기준)",title_text_color = '#FFFFFF',height = 10,color_mapper=color_mapper, label_standoff=12, background_fill_color="#212529",major_label_text_color = "#FFFFFF")
p.add_layout(color_bar, 'below')

for data, name, color in zip([bb, cc], ["주간산규확진자", "승차인원"], Spectral4):
    df = pd.DataFrame(data)
    df['주간시작일자'] = pd.to_datetime(df['주간시작일자'])
    p.line(df['주간시작일자'], df['신규확진자'], line_width=3, color=color, alpha=1,
           muted_color=color, muted_alpha=0.2, legend_label=name)

p.legend.location = "top_left"
p.legend.click_policy="mute"

layout = layout([[toggle],[p]])
# layout = bl([p], [toggle])
curdoc().add_root(layout)

show(layout)

In [84]:
del doc

NameError: name 'doc' is not defined